In [14]:
import tkinter as tk
from tkinter import ttk
from PIL import Image
from PIL import ImageTk
import tkinter.filedialog as tkf
import cv2
import numpy as np
import imutils
import math
import glob
import time

count = 1
mtx = 0
dist = 0
retx = False
newcameramtx = 0
mean_error = 0
ratio = 1
thresval = 150
foundstatus = 0
foundcount = 0
shape = ""
size = 0
result = []
CalibState = 0
HoughParam = [1, 10, 150, 15, 0, 450]

# get image from the camera
video = cv2.VideoCapture(1)

# root window
root = tk.Tk()
root.title("Shape Measurement Program")
#root.geometry("1080x720")
root.state("zoomed")
root.resizable(False, False)

# create video streaming panel
Stream = tk.Frame()
Stream.grid(row = 0, column = 0)
Calib = tk.Frame()
Calib.grid(row = 0, column = 0)

# create a notebook
s = ttk.Style()
s.configure('TNotebook.Tab', font = ("Helvetica", 12))
notebook = ttk.Notebook(root)
notebook.grid(row = 0, column = 1, sticky = "NW")

# create frames for notebook
Measurement = ttk.Frame(notebook, width = 400, height = 824)
Calibration = ttk.Frame(notebook, width = 400, height = 824)
Parameter = ttk.Frame(notebook, width = 400, height = 824)

# Measurement tab
# define columns
columns = ("No", "Shape Type", "Size")

tree = ttk.Treeview(Measurement, columns = columns, show = "headings", height = 30)

# define headings
tree.heading("No", text = "No")
tree.column("No", width = 40, anchor = tk.CENTER)
tree.heading("Shape Type", text = "Shape Type")
tree.column("Shape Type", width = 150, anchor = tk.CENTER)
tree.heading("Size", text = "Size (mm)")
tree.column("Size", width = 200, anchor = tk.CENTER)
tree.grid(row = 0, column = 0, sticky='NSEW')

# add a scrollbar
scrollbar = ttk.Scrollbar(Measurement, orient = tk.VERTICAL, command = tree.yview)
tree.configure(yscroll = scrollbar.set)
scrollbar.grid(row = 0, column = 1, sticky = 'NS')

# Calibration tab
CMLb = tk.Label(Calibration, text = "Camera Matrix :", font = ("Helvetica", 12))
CMLb.grid(row = 0, column = 0, sticky = "E", padx = 10, pady = 10)
CM_ValLb = tk.Label(Calibration, font = ("Helvetica", 12))
CM_ValLb.grid(row = 0, column = 1, columnspan = 2, sticky = "W", padx = 10, pady = 10)
DCLb = tk.Label(Calibration, text = "Distortion Coefficient :", font = ("Helvetica", 12))
DCLb.grid(row = 1, column = 0, sticky = "E", padx = 10, pady = 10)
DC_ValLb = tk.Label(Calibration, font = ("Helvetica", 12))
DC_ValLb.grid(row = 1, column = 1, columnspan = 2, sticky = "W", padx = 10, pady = 10)
RatioLb= tk.Label(Calibration, text = "Ratio :", font = ("Helvetica", 12))
RatioLb.grid(row = 2, column = 0, sticky = "E", padx = 10, pady = 10)
Ratio_ValLb = tk.Label(Calibration, font = ("Helvetica", 12))
Ratio_ValLb.grid(row = 2, column = 1, sticky = "W", padx = 10, pady = 10)
Ratio_UnitLb = tk.Label(Calibration, text = "pixel/mm", font = ("Helvetica", 12))
Ratio_UnitLb.grid(row = 2, column = 2, sticky = "W", padx = 10, pady = 10)
ErrLb= tk.Label(Calibration, text = "Total Error :", font = ("Helvetica", 12))
ErrLb.grid(row = 3, column = 0, sticky = "E", padx = 10, pady = 10)
Err_ValLb = tk.Label(Calibration, font = ("Helvetica", 12))
Err_ValLb.grid(row = 3, column = 1, columnspan = 2, sticky = "W", padx = 10, pady = 10)
CalibBt = tk.Button(Calibration, text = "Calibrate", font = ("Helvetica", 12), width = 10)
CalibBt.grid(row = 4, column = 1, sticky = "NSEW", padx = 10, pady = 10)
ImgLb = tk.Label(Calibration, text = "Image :", font = ("Helvetica", 12))
ImgLb.grid(row = 5, column = 0, sticky = "E", padx = 10, pady = 10)
Img_NoLb = tk.Label(Calibration, text = str(count - 1), font = ("Helvetica", 12))
Img_NoLb.grid(row = 5, column = 1, columnspan = 2, sticky = "W", padx = 10, pady = 10)
PtSizeLb = tk.Label(Calibration, text = "Pattern Size (VxH) :", font = ("Helvetica", 12))
PtSizeLb.grid(row = 6, column = 0, sticky = "E", padx = 10, pady = 10)
PtSizeVEt = tk.Entry(Calibration, state = "disable")
PtSizeVEt.grid(row = 6, column = 1, sticky = "E", padx = 10, pady = 10)
PtSizeHEt = tk.Entry(Calibration, state = "disable")
PtSizeHEt.grid(row = 6, column = 2, sticky = "E", padx = 10, pady = 10)
RlSizeLb = tk.Label(Calibration, text = "Real Size (mm) :", font = ("Helvetica", 12))
RlSizeLb.grid(row = 7, column = 0, sticky = "E", padx = 10, pady = 10)
RlSizeEt = tk.Entry(Calibration, state = "disable")
RlSizeEt.grid(row = 7, column = 1, sticky = "E", padx = 10, pady = 10)
CaptureBt = tk.Button(Calibration, text = "Capture", font = ("Helvetica", 12), width = 10, state = "disable")
CaptureBt.grid(row = 8, column = 0, sticky = "NSEW", padx = 10, pady = 10)
FinishBt = tk.Button(Calibration, text = "Finish", font = ("Helvetica", 12), width = 10, state = "disable")
FinishBt.grid(row = 8, column = 1, sticky = "NSEW", padx = 10, pady = 10)
CancelBt = tk.Button(Calibration, text = "Cancel", font = ("Helvetica", 12), width = 10, state = "disable")
CancelBt.grid(row = 8, column = 2, sticky = "NSEW", padx = 10, pady = 10)

# Parameter tab
thresLb = tk.Label(Parameter, text = "Threshold :", font = ("Helvetica", 12))
thresLb.grid(row = 0, column = 0, sticky = "E", padx = 10, pady = 10)
thres_valEt = tk.Entry(Parameter)
thres_valEt.grid(row = 0, column = 1, sticky = "E", padx = 10, pady = 10)
dpLb = tk.Label(Parameter, text = "dp :", font = ("Helvetica", 12))
dpLb.grid(row = 1, column = 0, sticky = "E", padx = 10, pady = 10)
dp_valEt = tk.Entry(Parameter)
dp_valEt.grid(row = 1, column = 1, sticky = "E", padx = 10, pady = 10)
minDistLb = tk.Label(Parameter, text = "minDist :", font = ("Helvetica", 12))
minDistLb.grid(row = 2, column = 0, sticky = "E", padx = 10, pady = 10)
minDist_valEt = tk.Entry(Parameter)
minDist_valEt.grid(row = 2, column = 1, sticky = "E", padx = 10, pady = 10)
Param1Lb = tk.Label(Parameter, text = "Param1 :", font = ("Helvetica", 12))
Param1Lb.grid(row = 3, column = 0, sticky = "E", padx = 10, pady = 10)
Param1_valEt = tk.Entry(Parameter)
Param1_valEt.grid(row = 3, column = 1, sticky = "E", padx = 10, pady = 10)
Param2Lb = tk.Label(Parameter, text = "Param2 :", font = ("Helvetica", 12))
Param2Lb.grid(row = 4, column = 0, sticky = "E", padx = 10, pady = 10)
Param2_valEt = tk.Entry(Parameter)
Param2_valEt.grid(row = 4, column = 1, sticky = "E", padx = 10, pady = 10)
minRadiusLb = tk.Label(Parameter, text = "minRadius :", font = ("Helvetica", 12))
minRadiusLb.grid(row = 5, column = 0, sticky = "E", padx = 10, pady = 10)
minRadius_valEt = tk.Entry(Parameter)
minRadius_valEt.grid(row = 5, column = 1, sticky = "E", padx = 10, pady = 10)
maxRadiusLb = tk.Label(Parameter, text = "maxRadius :", font = ("Helvetica", 12))
maxRadiusLb.grid(row = 6, column = 0, sticky = "E", padx = 10, pady = 10)
maxRadius_valEt = tk.Entry(Parameter)
maxRadius_valEt.grid(row = 6, column = 1, sticky = "E", padx = 10, pady = 10)

# add frame to notebook
notebook.add(Measurement, text = "Measurement")
notebook.add(Calibration, text = "Calibration")
notebook.add(Parameter, text = "Parameter")

# create label for streaming video
Original = tk.Label(Stream)
Original.grid(row = 0, column = 0, sticky = "NSEW")
Thresholding = tk.Label(Stream)
Thresholding.grid(row = 0, column = 1, sticky = "NSEW")
Detection = tk.Label(Stream)
Detection.grid(row = 1, column = 0, sticky = "NSEW")
Measurement = tk.Label(Stream)
Measurement.grid(row = 1, column = 1, sticky = "NSEW")
OriginalL = tk.Label(Stream, text = "Original", font = ("Helvetica", 16), bg = "red", fg = "white")
OriginalL.place(x = 2, y = 2)
ThresholdingL = tk.Label(Stream, text = "Thresholding", font = ("Helvetica", 16), bg = "red", fg = "white")
ThresholdingL.place(x = 2 + 544 + 4, y = 2)
DetectionL = tk.Label(Stream, text = "Detection", font = ("Helvetica", 16), bg = "red", fg = "white")
DetectionL.place(x = 2, y = 2 + 408 + 4)
MeasurementL = tk.Label(Stream, text = "Measurement", font = ("Helvetica", 16), bg = "red", fg = "white")
MeasurementL.place(x = 2 + 544 + 4, y = 2 + 408 + 4)

#FSLB = tk.Label(Calibration, text = "Measurement")
#FSLB.grid(row = 9, column = 0, sticky='NSEW')

def show_Stream(): 
    global count
    global mtx
    global dist
    global retx
    global newcameramtx
    global mean_error
    global ratio
    global thresval
    global foundstatus
    global foundcount
    global foundstatus
    global shape
    global size
    global result
    global CalibState
    global HoughParam
    
    # get image from the camera
    cX, cY =(0, 0)
    ret, frame = video.read()
    frame = cv2.flip(frame, -1)
    if retx == True:
        frame = cv2.undistort(frame, mtx, dist, None, newcameramtx)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detection = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    measurement = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray_img, (5,5), 0)
    threshold = cv2.threshold(blurred, thresval, 255, cv2.THRESH_BINARY)[1]
    conts = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    conts = imutils.grab_contours(conts)
    circle = cv2.HoughCircles(threshold,cv2.HOUGH_GRADIENT, HoughParam[0], HoughParam[1],
                              param1 = HoughParam[2], param2 = HoughParam[3], minRadius = HoughParam[4], 
                              maxRadius = HoughParam[5])
    line = cv2.line(detection, (int(detection.shape[1]/2), 0), 
                                     (int(detection.shape[1]/2), int(detection.shape[0])), (0, 255, 0), thickness=2)
    
    #draw bounding box
    for c in conts:
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        M = cv2.moments(c)
        if M["m00"] != 0:
            cX = int((M["m10"] / M["m00"]))
            cY = int((M["m01"] / M["m00"]))
        if len(approx) == 3:
            cv2.drawContours(measurement, [c], -1, (255,0,0), 2)
            cv2.circle(detection, (cX, cY), 2, (0,0,255), 3)
            cv2.circle(measurement, (cX, cY), 2, (0,0,255), 3)
            p1, p2, p3 = approx
            l1 = math.fabs(round(math.sqrt((p2[0][0] - p1[0][0])**2 + (p2[0][1] - p1[0][1])**2)/ratio,2))
            l2 = math.fabs(round(math.sqrt((p3[0][0] - p2[0][0])**2 + (p3[0][1] - p2[0][1])**2)/ratio,2))
            l3 = math.fabs(round(math.sqrt((p3[0][0] - p1[0][0])**2 + (p3[0][1] - p1[0][1])**2)/ratio,2))
            size = (l1, l2, l3)
            shape = "triangle"
            p = ((p2-p1)/2)+p1
            cv2.putText(measurement, str(l1), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l1), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            p = ((p3-p2)/2)+p2
            cv2.putText(measurement, str(l2), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l2), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            p = ((p3-p1)/2)+p1
            cv2.putText(measurement, str(l3), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l3), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
                
            
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            cv2.drawContours(measurement, [c], -1, (255,0,0), 2)
            cv2.circle(detection, (cX, cY), 2, (0,0,255), 3)
            cv2.circle(measurement, (cX, cY), 2, (0,0,255), 3)
            p1, p2, p3, p4 = approx
            l1 = math.fabs(round(math.sqrt((p2[0][0] - p1[0][0])**2 + (p2[0][1] - p1[0][1])**2)/ratio,2))
            l2 = math.fabs(round(math.sqrt((p3[0][0] - p2[0][0])**2 + (p3[0][1] - p2[0][1])**2)/ratio,2))
            l3 = math.fabs(round(math.sqrt((p4[0][0] - p3[0][0])**2 + (p4[0][1] - p3[0][1])**2)/ratio,2))
            l4 = math.fabs(round(math.sqrt((p4[0][0] - p1[0][0])**2 + (p4[0][1] - p1[0][1])**2)/ratio,2))
            size = (l1, l2, l3, l4)
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
            p = ((p2-p1)/2)+p1
            cv2.putText(measurement, str(l1), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l1), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            p = ((p3-p2)/2)+p2
            cv2.putText(measurement, str(l2), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l2), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            p = ((p4-p3)/2)+p3
            cv2.putText(measurement, str(l3), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l3), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            p = ((p4-p1)/2)+p1
            cv2.putText(measurement, str(l4), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l4), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            
        # if the shape is a pentagon, it will have 5 vertices 
        elif len(approx) == 5:
            cv2.drawContours(measurement, [c], -1, (255,0,0), 2)
            cv2.circle(detection, (cX, cY), 2, (0,0,255), 3)
            cv2.circle(measurement, (cX, cY), 2, (0,0,255), 3)
            p1, p2, p3, p4, p5 = approx
            l1 = math.fabs(round(math.sqrt((p2[0][0] - p1[0][0])**2 + (p2[0][1] - p1[0][1])**2)/ratio,2))
            l2 = math.fabs(round(math.sqrt((p3[0][0] - p2[0][0])**2 + (p3[0][1] - p2[0][1])**2)/ratio,2))
            l3 = math.fabs(round(math.sqrt((p4[0][0] - p3[0][0])**2 + (p4[0][1] - p3[0][1])**2)/ratio,2))
            l4 = math.fabs(round(math.sqrt((p5[0][0] - p4[0][0])**2 + (p5[0][1] - p4[0][1])**2)/ratio,2))
            l5 = math.fabs(round(math.sqrt((p5[0][0] - p1[0][0])**2 + (p5[0][1] - p1[0][1])**2)/ratio,2))
            size = (l1, l2, l3, l4, l5)
            shape = "pentagon"
            p = ((p2-p1)/2)+p1
            cv2.putText(measurement, str(l1), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l1), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            p = ((p3-p2)/2)+p2
            cv2.putText(measurement, str(l2), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l2), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            p = ((p4-p3)/2)+p3
            cv2.putText(measurement, str(l3), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l3), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            p = ((p5-p4)/2)+p4
            cv2.putText(measurement, str(l4), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l4), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            p = ((p5-p1)/2)+p1
            cv2.putText(measurement, str(l4), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(l4), (int(p[0][0]),int(p[0][1])), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)
            
    # otherwise, we assume the shape is a circle
    if circle is not None:
        circle = np.round(circle[0, :]).astype("int")
        shape = "circle"
        for (cX,cY,r) in circle:
            size = math.fabs(round(r/ratio,2))
            # draw the outer circle
            cv2.circle(measurement,(cX,cY),r,(255,0,0),2)
            # draw the center of the circle
            cv2.circle(detection,(cX,cY),2,(0,0,255),3)
            cv2.circle(measurement,(cX,cY),2,(0,0,255),3)
            cv2.putText(measurement, str(size), (cX,cY+20), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 5)
            cv2.putText(measurement, str(size), (cX,cY+20), cv2.FONT_HERSHEY_SIMPLEX,0.5, (255, 255, 255), 2)

######################################################################################
    # check part detection 
    check = []
    for i in range(detection.shape[0]):
        check.append(detection[i, int(detection.shape[1]/2)])
        #print(i)
    #print(check)
    
    if(np.any(check != np.array([0,255,0]))):
        if(foundstatus == 0):
            #print("found")
            foundstatus = 1
            foundcount += 1
            tree.insert('', foundcount - 1, values = (foundcount, shape, size))
            result.append([foundcount, shape, size])
            #time.sleep(2)

    else:   
        #print("not found")
        foundstatus = 0
    #FSLB.configure(text = foundstatus)
######################################################################################

    # resize image
    #print('Original Dimensions : ',image.shape)
    scale_percent = 85 # percent of original size
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    threshold = cv2.resize(threshold, dim, interpolation = cv2.INTER_AREA)
    detection = cv2.resize(detection, dim, interpolation = cv2.INTER_AREA)
    measurement = cv2.resize(measurement, dim, interpolation = cv2.INTER_AREA)
    #print('Resized Dimensions : ', image.shape)
    
    # convert the images to PIL format...
    image = Image.fromarray(image)
    threshold = Image.fromarray(threshold)
    detection = Image.fromarray(detection)
    measurement = Image.fromarray(measurement)
    
    # ...and then to ImageTk format
    image = ImageTk.PhotoImage(image)
    threshold = ImageTk.PhotoImage(threshold)
    detection = ImageTk.PhotoImage(detection)
    measurement = ImageTk.PhotoImage(measurement)
    
    # update image in label
    Original.image = image
    Thresholding.image = threshold
    Detection.image = detection
    Measurement.image = measurement
    Original.configure(image = image)
    if(CalibState == 0):
        Thresholding.configure(image = threshold)
        Detection.configure(image = detection)
        Measurement.configure(image = measurement)
    Stream.after(1, show_Stream)
    Img_NoLb.configure(text = str(count - 1))
    CM_ValLb.configure(text = str(np.around(mtx, 2)))
    DC_ValLb.configure(text = str(np.around(dist, 2)))
    Ratio_ValLb.configure(text = str(np.around(ratio, 2)))
    Err_ValLb.configure(text = str(np.around(mean_error, 2)))
    #print(root.winfo_height(), root.winfo_width(), Stream.winfo_height(), Stream.winfo_width())
    
def CalibBt_Event(event):
    global CalibState
    if CalibBt["state"] == "active":
        print("CalibBt")
        CalibState = 1
    
    CalibBt["state"] = "disable"
    CaptureBt["state"] = "normal"
    FinishBt["state"] = "normal"
    CancelBt["state"] = "normal"
    RlSizeEt["state"] = "normal"
    PtSizeVEt["state"] = "normal"
    PtSizeHEt["state"] = "normal"

def CaptureBt_Event(event): 
    global count
    
    if CaptureBt["state"] == "active":
        print("CaptureBt")
        fname = ""
        ret, frame = video.read()
        frame = cv2.flip(frame,-1)
        fname = "src/calib/calib" + str(count) + ".png"
        print(cv2.imwrite(fname, frame))
        count = count + 1
    
    
def FinishBt_Event(event):
    global count
    global mtx
    global dist
    global retx
    global newcameramtx
    global roi
    global mean_error
    global ratio
    global CalibState
    
    if FinishBt["state"] == "active":
        print("FinishBt")
        realsize = float(RlSizeEt.get())
        V = int(PtSizeVEt.get())
        H = int(PtSizeHEt.get())
        print("realsize : ", realsize)
        count = 1
        patternsize = (V, H)
        print("patternsize :", patternsize)

        # termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        objp = np.zeros((patternsize[0]*patternsize[1],3), np.float32)
        objp[:,:2] = np.mgrid[0:patternsize[0],0:patternsize[1]].T.reshape(-1,2)
        objp[:,:2] *= realsize

        # Arrays to store object points and image points from all the images.
        objpoints = [] # 3d point in real world space
        imgpoints = [] # 2d points in image plane.

        while True:
            frame = cv2.imread("src/calib/calib" + str(count) + ".png")
            if frame is None:
                break
            h, w = frame.shape[:2]
            gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            retx, corners = cv2.findChessboardCorners(gray_img, patternsize, None)
            if retx == True:
                objpoints.append(objp)
                corners2 = cv2.cornerSubPix(gray_img, corners, (11,11), (-1,-1), criteria)
                if corners2[0, 0][0] > corners2[patternsize[0]+1, 0][0]: # flip points if it detects incorrect direction
                    corners2 = np.flip(corners2, 1)
                    corners2 = np.flip(corners2, 0)
                imgpoints.append(corners2)
            count = count + 1
        #print("coners :", corners)
        #print("objpoints :", objpoints)
        #print("coners2 :", corners2)
        #print("imgpoint :", imgpoints)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray_img.shape[::-1], None, None)
        print("calibrateCamera :", retx)
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
        print(roi)
        for i in range(len(objpoints)):
            imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
            error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
            mean_error += error
        mean_error = mean_error/len(objpoints)
        #print(newcameramtx)
        p0 = 32
        p1 = 39
        x1 = corners2[p0, 0][0]
        y1 = corners2[p0, 0][1]
        x2 = corners2[p1, 0][0]
        y2 = corners2[p1, 0][1]
        distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        ratio = distance / realsize
        CalibState = 0
    
    CalibBt["state"] = "normal"
    CaptureBt["state"] = "disable"
    FinishBt["state"] = "disable"
    CancelBt["state"] = "disable"
    RlSizeEt["state"] = "disable"
    PtSizeVEt["state"] = "disable"
    PtSizeHEt["state"] = "disable"
    
def CancelBt_Event(event):
    global count
    global CalibState
    
    if CancelBt["state"] == "active":
        print("CancelBt")
        count = 1
        CalibState = 0
    
    CalibBt["state"] = "normal"
    CaptureBt["state"] = "disable"
    FinishBt["state"] = "disable"
    CancelBt["state"] = "disable"
    RlSizeEt["state"] = "disable"
    PtSizeVEt["state"] = "disable"
    PtSizeHEt["state"] = "disable"

def thres_valEt_Event(event):
    global thresval
    thresval = int(thres_valEt.get())
    if(thresval < 0):
        thresval = 0
    if(thresval > 255):
        thresval = 255
        
def dp_valEt_Event(event):
    global HoughParam
    HoughParam[0] = int(dp_valEt.get())
    
def minDist_valEt_Event(event):
    global HoughParam
    HoughParam[1] = int(minDist_valEt.get())
    
def Param1_valEt_Event(event):
    global HoughParam
    HoughParam[2] = int(Param1_valEt.get())
    
def Param2_valEt_Event(event):
    global HoughParam
    HoughParam[3] = int(Param2_valEt.get())
    
def minRadius_valEt_Event(event):
    global HoughParam
    HoughParam[4] = int(minRadius_valEt.get())
    
def maxRadius_valEt_Event(event):
    global HoughParam
    HoughParam[5] = int(maxRadius_val.get())
    
# event
CalibBt.bind('<ButtonRelease>', CalibBt_Event)
CaptureBt.bind('<ButtonRelease>', CaptureBt_Event)
FinishBt.bind('<ButtonRelease>', FinishBt_Event)
CancelBt.bind('<ButtonRelease>', CancelBt_Event)
thres_valEt.bind('<KeyRelease>', thres_valEt_Event)
dp_valEt.bind('<KeyRelease>', dp_valEt_Event)
minDist_valEt.bind('<KeyRelease>', minDist_valEt_Event)
Param1_valEt.bind('<KeyRelease>', Param1_valEt_Event)
Param2_valEt.bind('<KeyRelease>', Param2_valEt_Event)
minRadius_valEt.bind('<KeyRelease>', minRadius_valEt_Event)
maxRadius_valEt.bind('<KeyRelease>', maxRadius_valEt_Event)

# load previous parameters
mtx = np.load("src/camera_mat.npy")
dist = np.load("src/dist_coefs.npy")
mean_error = np.load("src/mean_error.npy")
retx = np.load("src/calib_status.npy")
newcameramtx = np.load("src/newcameramtx.npy")
ratio = np.load("src/ratio.npy")
thresval = np.load("src/thresval.npy")
HoughParam = np.load("src/houghparam.npy")

show_Stream()
root.mainloop()
video.release()

# save parameters
np.save("src/camera_mat.npy", mtx)
np.save("src/dist_coefs.npy", dist)
np.save("src/mean_error.npy", mean_error)
np.save("src/calib_status.npy", retx)
np.save("src/newcameramtx.npy", newcameramtx)
np.save("src/ratio.npy", ratio)
np.save("src/thresval.npy", thresval)
np.save("src/houghparam.npy", HoughParam)
np.savetxt("src/result.txt", result, delimiter = ',', fmt = '%s')

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [15]:
print(result)

[[1, 'rectangle', (78.21, 50.22, 79.25, 50.54)], [2, 'rectangle', (77.61, 50.73, 79.34, 50.6)], [3, 'rectangle', (50.14, 78.51, 50.79, 78.95)], [4, 'rectangle', (77.67, 50.24, 79.09, 50.36)]]
